In [38]:
import pandas as pd 
import numpy as np 

dataframe = pd.read_excel('15-Points.xlsx')
dataframe.head()

,x,y,City
0,5.500000e-08,9.860000e-09,1
1,-2.887330e+01,-7.980000e-08,2
2,-7.929160e+01,-2.140330e+01,3
3,-1.465770e+01,-4.338960e+01,4
4,-6.474730e+01,2.189820e+01,5


In [39]:
import math
def distance_matrix(df):
    # calculate distance matrix
    dist_matrix = [[math.sqrt((df['x'][i] - df['x'][j])**2 + (df['y'][i] - df['y'][j])**2)
                  for j in range(len(df))] for i in range(len(df))]
    return dist_matrix

In [40]:
dm = distance_matrix(dataframe)

In [41]:
import random

class Chromosome:
    def __init__(self, city_order):
        self.city_order = city_order
        self.cost = None
        self.fitness = None
    
    def create_random_chromosome(city_list):
        city_order = city_list[:]
        random.shuffle(city_order)
        return Chromosome(city_order)
    
    def calculate_cost(self, distance_matrix):
        cost = 0
        for i in range(len(self.city_order) - 1):
            city1 = self.city_order[i]
            city2 = self.city_order[(i + 1)]
            cost += distance_matrix[city1][city2]
        city1 = self.city_order[-1]
        city2 = self.city_order[0]
        cost += distance_matrix[city1][city2]
        self.cost = cost
    
    def calculate_fitness(self):
        self.fitness = 1 / self.cost if self.cost is not None and self.cost != 0 else np.Inf
    
    def crossover(self, other_chromosome):
        crossover_point = random.randint(0, len(self.city_order) - 1)
        
        offspring1 = self.city_order[:crossover_point]
        for city in other_chromosome.city_order:
            if city not in offspring1:
                offspring1.append(city)
        
        offspring2 = other_chromosome.city_order[:crossover_point]
        for city in self.city_order:
            if city not in offspring2:
                offspring2.append(city)
        
        return Chromosome(offspring1), Chromosome(offspring2)
        
    def mutate(self, mutation_rate):
        
        if random.random() < mutation_rate:
            city1_index, city2_index = random.sample(range(len(self.city_order)), 2)
            self.city_order[city1_index], self.city_order[city2_index] = self.city_order[city2_index], self.city_order[city1_index]
                
        
    def __repr__(self):
        return f"Chromosome(city_order={self.city_order}, cost={self.cost}, fitness={self.fitness})"


In [42]:
city_list = [0,1,2,3,4]
c1 = Chromosome.create_random_chromosome(city_list)
c1.calculate_cost(dm)
c1.calculate_fitness()
print(c1)

Chromosome(city_order=[3, 2, 0, 4, 1], cost=306.4391643808877, fitness=0.0032632904544702807)


In [63]:
def genetic_algorithm(distance_matrix, max_iterations, num_population, mutation_rate, crossover_ratio, mutation_ratio, elite_ratio):
          
    elite_number = int(num_population * elite_ratio)
    crossover_number = int(num_population * crossover_ratio)
    mutation_number = int(num_population * mutation_ratio)
    
    population = [Chromosome.create_random_chromosome(list(range(len(distance_matrix)))) for _ in range(num_population)]
    
    for chromosome in population:
        chromosome.calculate_cost(distance_matrix)
        chromosome.calculate_fitness()
    
    best_chromosome = min(population, key=lambda x: x.cost)
    print(f"Iteration 0: Best distance = {best_chromosome.cost}, Best path = {best_chromosome.city_order}")
    
    for iteration in range(1, max_iterations+1):
        parents = random.sample(population, crossover_number)
        offspring = []
        for i in range(0, crossover_number, 2):
            offspring1, offspring2 = parents[i].crossover(parents[i+1])
            offspring1.calculate_cost(distance_matrix)
            offspring1.calculate_fitness()
            offspring2.calculate_cost(distance_matrix)
            offspring2.calculate_fitness()
            offspring.append(offspring1)
            offspring.append(offspring2)
        
        mutated_offspring = random.sample(offspring, mutation_number)
        for chromosome in mutated_offspring:
            chromosome.mutate(mutation_rate)
            chromosome.calculate_cost(distance_matrix)
            chromosome.calculate_fitness()
            offspring.append(chromosome)
                        
        elite_population = sorted(population, key=lambda x: x.cost)[:elite_number]
                
        combined_population = elite_population + offspring
        population = sorted(combined_population, key=lambda x: x.cost, reverse=False)[:num_population]
        
        for chromosome in population:
            chromosome.calculate_cost(distance_matrix)
            chromosome.calculate_fitness()
        
        best_chromosome = min(population, key=lambda x: x.cost)
        print(f"Iteration {iteration}: Best distance = {best_chromosome.cost}, Best path = {best_chromosome.city_order}")
    
    return best_chromosome


In [64]:
genetic_algorithm(dm,2000,200,0.02,0.7,0.1,0.2)

Iteration 0: Best distance = 477.56134984402354, Best path = [1, 9, 7, 14, 8, 6, 4, 10, 5, 11, 13, 2, 3, 0, 12]
Iteration 1: Best distance = 477.56134984402354, Best path = [1, 9, 7, 14, 8, 6, 4, 10, 5, 11, 13, 2, 3, 0, 12]
Iteration 2: Best distance = 477.56134984402354, Best path = [1, 9, 7, 14, 8, 6, 4, 10, 5, 11, 13, 2, 3, 0, 12]
Iteration 3: Best distance = 449.10182080724655, Best path = [4, 14, 6, 2, 13, 7, 9, 3, 5, 11, 10, 12, 0, 1, 8]
Iteration 4: Best distance = 449.10182080724655, Best path = [4, 14, 6, 2, 13, 7, 9, 3, 5, 11, 10, 12, 0, 1, 8]
Iteration 5: Best distance = 449.10182080724655, Best path = [4, 14, 6, 2, 13, 7, 9, 3, 5, 11, 10, 12, 0, 1, 8]
Iteration 6: Best distance = 406.2939898031711, Best path = [7, 5, 3, 9, 2, 13, 11, 4, 14, 6, 8, 1, 12, 0, 10]
Iteration 7: Best distance = 406.2939898031711, Best path = [7, 5, 3, 9, 2, 13, 11, 4, 14, 6, 8, 1, 12, 0, 10]
Iteration 8: Best distance = 406.2939898031711, Best path = [7, 5, 3, 9, 2, 13, 11, 4, 14, 6, 8, 1, 12, 0,

Iteration 102: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 103: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 104: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 105: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 106: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 107: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 108: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 109: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 110: Best distance = 308.9023920870121, Best path = [4, 14, 6, 2, 11, 13, 9, 7, 5, 3, 

Iteration 227: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 228: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 229: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 230: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 231: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 232: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 233: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 234: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 235: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 

Iteration 360: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 361: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 362: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 363: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 364: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 365: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 366: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 367: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 8]
Iteration 368: Best distance = 289.40917445687427, Best path = [14, 4, 6, 2, 11, 13, 9, 

Iteration 466: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 467: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 468: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 469: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 470: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 471: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 472: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 473: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 474: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 1

...

Iteration 1610: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1611: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1612: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1613: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1614: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1615: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1616: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1617: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1618: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7

Iteration 1744: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1745: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1746: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1747: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1748: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1749: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1750: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1751: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1752: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7

Iteration 1886: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1887: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1888: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1889: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1890: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1891: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1892: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1893: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1894: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7

Iteration 1958: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1959: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1960: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1961: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1962: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1963: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1964: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1965: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14]
Iteration 1966: Best distance = 284.3810904080332, Best path = [8, 4, 6, 2, 11, 13, 9, 7

Chromosome(city_order=[8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10, 0, 12, 1, 14], cost=284.3810904080332, fitness=0.003516408206203826)